In [7]:
import sys
sys.path.append('../')

In [8]:
import AbreivBot.Functions as functions
import AbreivBot.Models.BaseModels as base_model

In [9]:
import pandas as pd
import numpy as np
import string

In [10]:
import tensorflow as tf
devices = tf.config.list_physical_devices('GPU')

In [11]:
tf.config.set_logical_device_configuration(devices[0], [tf.config.LogicalDeviceConfiguration(memory_limit=300)])

In [12]:
abbrev = pd.read_csv('TrainAndTest/traindata/cleaned_abbreviations.csv')
abbrev = abbrev[abbrev.columns[1:]]

In [13]:
abbrev['parse_ylabel'] = abbrev.apply(lambda x: ''.join(functions.encode_ylabel(x['Abbreviation'], x['Word'])), axis=1)
abbrev['WordLength'] = abbrev['Word'].apply(lambda x: len(x))

In [14]:
abbrev.head()

,Abbreviation,Word,parse_ylabel,WordLength
0,A.D.,Anno Domini,A.@@@D.@@@@,11
1,A.V.,Authorized Version,A.@@@@@@@@@V.@@@@@,18
2,Abbrev.,abbreviation,Abbrev.@@@@@,12
3,Abbrev.,abbreviations,Abbrev.@@@@@@,13
4,Abd.,Aberdeen,Ab@@d.@@,8


In [15]:
possible_chars = string.ascii_letters + string.digits + ' .@'
possible_chars

'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789 .@'

In [16]:
model = base_model.BaseModel(possible_chars)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.build(input_shape=(None, None, len(possible_chars)))

2023-04-13 05:30:02.204925: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-04-13 05:30:02.205097: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-04-13 05:30:02.205205: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [ ]:
train = abbrev.sample(frac=0.8, random_state=1337)
test = abbrev.drop(train.index)

In [ ]:
import AbreivBot.TrainModel as train_model
import AbreivBot.TestModel as test_model

In [ ]:
history = train_model.train(model, train, possible_chars, verbose=0)

In [17]:
model.summary()

Model: "base_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  65        
                                                                 
 flatten (Flatten)           multiple                  0 (unused)
                                                                 
 lstm (LSTM)                 multiple                  34060     
                                                                 
 dense (Dense)               multiple                  4290      
                                                                 
Total params: 38,415
Trainable params: 38,415
Non-trainable params: 0
_________________________________________________________________


In [18]:
train = abbrev.sample(frac=0.8, random_state=1337)
test = abbrev.drop(train.index)

In [19]:
import AbreivBot.TrainModel as train_model
import AbreivBot.TestModel as test_model

In [20]:
history = train_model.train(model, train, possible_chars, verbose=0)

2023-04-13 05:30:06.135342: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-04-13 05:30:06.136906: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-04-13 05:30:06.138102: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [21]:
results = test_model.test(model, test, possible_chars, verbose=0)

2023-04-13 05:31:54.609267: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [32,8,65]
	 [[{{node Placeholder/_1}}]]
2023-04-13 05:31:56.258931: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [21,13,65]
	 [[{{node Placeholder/_1}}]]
2023-04-13 05:32:00.485902: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [

In [22]:
results

{8: [2.639249563217163, 0.4453125],
 13: [2.537280797958374, 0.5457875728607178],
 10: [2.339392900466919, 0.5265306234359741],
 11: [2.958853244781494, 0.4733542203903198],
 14: [2.6864566802978516, 0.4920634925365448],
 9: [2.90810489654541, 0.4135802388191223],
 12: [2.8512160778045654, 0.48924732208251953],
 15: [2.8231637477874756, 0.4166666567325592],
 7: [2.6251120567321777, 0.41871920228004456],
 16: [3.8799452781677246, 0.4375],
 5: [3.027543067932129, 0.46666666865348816],
 20: [2.9079604148864746, 0.6499999761581421],
 19: [2.3682868480682373, 0.5263158082962036],
 6: [2.4516210556030273, 0.4444444477558136],
 4: [2.367194175720215, 0.5],
 22: [2.000842332839966, 0.6666666865348816],
 18: [3.441504716873169, 0.3333333432674408],
 17: [1.3112642765045166, 0.5882353186607361],
 21: [2.152961254119873, 0.6190476417541504]}

In [26]:
model.save_weights('../trained_models/base_model/base_model_04_13_2023')